# Data I/O, featurization and coordinate transforms in PyEMMA
Ingredients:
- Topology file: PDB
- Trajectory data: List of .XTC files

Issues:
- Ensure trajectory list (.xtc) and topology file (.pdb) have the same number of atoms (any difference produces errors!). 
    - !!!RECALL: GMX trjconv option 3 is "Protein-H" which is "protein minus H atoms"! 

In [25]:
topfile = '5kv7_nohho.pdb'      # define topology with waterless PDB
traj_list = 'md_0_1_nohho_prominh_calpha.xtc'     # 

The `fetch` function fetches the data from our servers. **Do not use `mdshare` for your own data!**

## Import PyEMMA & friends

In [26]:
import pyemma
import numpy as np
import matplotlib.pyplot as plt
plt.matplotlib.rcParams.update({'font.size': 16})

## The featurizer
All information for data processing (raw MD -> observable) is stored in a `Featurizer` object.

In [27]:
featurizer = pyemma.coordinates.featurizer(topfile)
featurizer

MDFeaturizer with features:
[, ...]

Features are simply added like this:
```python
featurizer.add_my_feature()
```
For example, we will add all heavy atom distances by first selecting heavy atoms

In [28]:
#heavy_atom_indices = featurizer.select_Heavy()      # Aha!!
sele = featurizer.select_Ca()     # Take in the backbone residues
#print(heavy_atom_indices)
print("The number of featurizer indices is",len(sele))

The number of featurizer indices is 164


... and by adding distances between them:

In [29]:
#featurizer.add_distances(sele, periodic=False)
featurizer.add_distances_ca()

We can add several different features; to find out which ones have been added, simply use `featurizer.describe()`.

In [30]:
#featurizer.describe()

There are some more handy methods that come with the featurizer:

In [31]:
featurizer.dimension()

13041

In [32]:
featurizer.pairs([1, 8, 18])

array([[ 1,  8],
       [ 1, 18],
       [ 8, 18]])

## loading featurized data
When dealing with datasets that fit into memory, we preferably load the data directly with
#### `load`

In [33]:
Y = pyemma.coordinates.load(traj_list, featurizer)

Alternatively, for high memory demands, the data can be streamed with
#### `source`

In [34]:
source = pyemma.coordinates.source(traj_list, featurizer)

The source object has some useful properties. e.g.

In [35]:
source.trajectory_lengths()

array([101])

We go on with the data in our memory, `Y`. Let's do a component-wise histogram plot of the loaded data:

In [36]:
fig, ax = plt.subplots(figsize=(10, 14))
pyemma.plots.plot_feature_histograms(np.concatenate(Y), 
                                     feature_labels=featurizer, 
                                     ax=ax)
ax.set_xlabel('heavy atom distance')
ax.set_title('distance histograms per dimension (normalized)');

IndexError: tuple index out of range

## Dimension reduction
The very high dimensional space can be transformed into a lower dimensional representation of the dynamics e.g. with TICA:

In [13]:
tica = pyemma.coordinates.tica(Y, lag=10, var_cutoff=0.95)

NameError: name 'Y' is not defined

`pyemma.coordinates.tica()` returns a TICA python object. We get the transformed data (`tics`) from it:

In [14]:
tics = tica.get_output()

NameError: name 'tica' is not defined

The TICA object contains useful properties such as the number of dimension that explain `var_cutoff` of the kinetic variance. The output data has the same shape.

In [15]:
tica.dimension()

NameError: name 'tica' is not defined

Let's visualize these two dimensions in a 2D histogram:

In [16]:
pyemma.plots.plot_free_energy(np.concatenate(tics)[:, 0], np.concatenate(tics)[:, 1])
plt.xlabel('TIC 1') 
plt.ylabel('TIC 2');

NameError: name 'tics' is not defined

## VAMP-scoring
We can use the VAMP-2 score e.g. to assess how many dimensions we should ideally take. We check for

In [17]:
dims = [1, 2, 3, 5]

To avoid overfitting, we perform cross validation:

In [18]:
def score_cv(data, dim):
    """Compute a cross-validated VAMP2 score.
    """
    # we temporarily suppress very short-lived progress bars
    from pyemma.util.contexts import settings
    with settings(show_progress_bars=False):
        
        scores = np.zeros(3)
        
        for n in range(3):
            vamp = pyemma.coordinates.vamp(
                [d for i, d in enumerate(data) if i != n], 
                lag=tica.lag, 
                dim=dim)
            
            scores[n] = vamp.score(data[n])
    return scores

In [19]:
fig, ax = plt.subplots(1, 1, figsize=(12, 3), sharey=True)

scores = []
errors = []

for dim in dims:
    torsions_scores = score_cv(Y, dim=dim)
    scores.append(torsions_scores.mean())
    errors.append(torsions_scores.std())

ax.bar([str(d) for d in dims], scores, yerr=errors)

ax.set_ylabel('VAMP2 score\n @ {} ps'.format(tica.lag))
ax.set_xlabel('# dimensions')

NameError: name 'Y' is not defined

We note that the VAMP-2 score is converged at 2 dimensions.

## Discretization / clustering
There are different ways of clustering the data, we use $k$-means here.

In [20]:
clustering = pyemma.coordinates.cluster_kmeans(tics, 
                                               k=75,
                                               stride=50,  # very large stride for presentation purposes only
                                               max_iter=30)

NameError: name 'tics' is not defined

As before, the clustering routine returns an object with several useful properties and methods. For example, let us visualize the cluster centers stored in `clustering.clustercenters`:

In [21]:
fig, ax = plt.subplots()
ax.plot(*clustering.clustercenters.T, 'ko')
pyemma.plots.plot_free_energy(*np.concatenate(tics).T, ax=ax)
ax.set_xlabel('$\Phi$ / rad') 
ax.set_ylabel('$\Psi$ / rad');

NameError: name 'clustering' is not defined

Most importantly, the clustering object contains the discrete trajectories that we need for later MSM estimation. Each frame in each trajectory gets assigned to one of the cluster centers here.

In [22]:
dtrajs = clustering.dtrajs

NameError: name 'clustering' is not defined

In [23]:
fig, ax = plt.subplots(2, 1, figsize=(15, 8), sharex=True)
b, e = 20400, 21100
ax[0].plot(tics[0][b:e, 0], alpha=.75, label='TIC 1')
ax[0].plot(tics[0][b:e, 1], alpha=.75, label='TIC 2')
ax[0].set_ylabel('TICA transformed data')
ax[0].legend()
ax[1].step(range(dtrajs[0][b:e].shape[0]), dtrajs[0][b:e])
ax[1].set_xlabel('time (steps)')
ax[1].set_ylabel('state')
fig.tight_layout()

NameError: name 'tics' is not defined

In [24]:
print(dtrajs[0][:25])

NameError: name 'dtrajs' is not defined